In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time

In [2]:
import cmath
 
class FFT:
    def __init__(self):
        self.pi = cmath.pi
    
    def convolution(self, g, h):
        l = len(g) + len(h)
        n = 1 << l.bit_length()
        
        g = g + [0] * (n - len(g))
        h = h + [0] * (n - len(h))
       
        gg = self.fft(g, n)
        hh = self.fft(h, n)
        ff = [gg[i] * hh[i] for i in range(n)]
 
        return self.inv_fft(ff, n)
 
    def fft(self, f, n, sgn = 1):
        if n == 1: return f
 
        f0 = [f[2 * i] for i in range(n // 2)]
        f1 = [f[2 * i + 1] for i in range(n // 2)]
        f0 = self.fft(f0, n // 2, sgn)
        f1 = self.fft(f1, n // 2, sgn)
 
        zeta = cmath.rect(1, sgn * self.pi * 2 / n)
        pow_zeta = 1
 
        for i in range(n):
            f[i] = f0[i % (n // 2)] + pow_zeta * f1[i % (n // 2)]
            pow_zeta *= zeta
 
        return f
 
    def inv_fft(self, f, n):
        ret = self.fft(f, n, -1)
        for i in range(n):
            ret[i] = (ret[i].real / n)
 
        return ret

In [ ]:
"""
方針
モノマーのみと反応させる　不活化したポリマーは再活性化しない(別に話が簡単になっていいけど)
モノマーとラジカルの重合確率は同じとする
"""

Time=3000
N=1*10**4
mol=10**6
init=mol/1000#

mono=[0 for i in range(2)] #モノマーだけ
mono[1]=mol

#ループ開始用
A=mono[:]+[0]*(N-len(mono))
B=[0.0]*N
C=[0.0]*N

fft=FFT()

k=0.1
r=0
threshold=init*k

dead=0
for step in range(Time):
    print(step)
    #開始剤の記述
    print("開始剤:%f" % (init))
    #k+=0.0001
    delta_init=k*init
    init-=delta_init
    init=max(init,0)
    B[0]+=delta_init
    
    #反応する分子数を決める
    SUM_A,SUM_B=sum(A),sum(B)
    x=1/2*(SUM_B/(SUM_A+SUM_B))#アタックを受けて殺されるラジカルの割合
    B_die=[x*B[i] for i in range(N)]
    B=[B[i]-B_die[i] for i in range(N)]
    
    SUM_B_die=sum(B_die)
    A_per=[A[i]/(SUM_A) for i in range(N)]
    #B_per=[B[i]/(SUM_A+SUM_B_die) for i in range(N)]
    B_die_per=[B_die[i]/(SUM_B_die) for i in range(N)]
    print("B_die_per:%f" % (sum(B_die_per)))

    #ここでB_dieとの整合性とれない？？？

    #BtoA=[B[i]*SUM_A/(SUM_A+SUM_B_die) for i in range(N)]
    #BtoB=[B[i]*SUM_B_die/(SUM_A+SUM_B_die) for i in range(N)]
    BtoA=[B[i]-B_die[i] for i in range(N)]
    BtoB=B_die[:]

    #FFTする　これはそれぞれの生成を考えるときに使う？
    delta_C=(fft.convolution(B_die_per,BtoB))
    delta_B=(fft.convolution(A_per,BtoA))

    dead_N=len(delta_C)-len(C)
    dead+=sum([delta_C[i]*i for i in range(N,N+dead_N)])+sum([delta_B[i]*i for i in range(N,N+dead_N)])
    print("残りモノマー:%f ラジカル反応数:%f 死ぬラジカル数%f 生まれたポリマー数%f" % (SUM_A,SUM_B,SUM_B_die,sum(delta_C)))#死ぬラジカル数とポリマー数が一緒じゃないとおかしくない？
    print("計算上死んだモノマー:%f" % (dead))
    
    #モノマー
    A[1]-=SUM_B-2*SUM_B_die
    #ラジカル
    B=[delta_B[i] if delta_B[i]>10**-4 else 0 for i in range(len(A))]
    #ポリマー
    C=[C[i]+delta_C[i] for i in range(len(A))]
    
    print('ポリマー量:%f' % (sum([A[i]*i for i in range(len(A))])+sum([B[i]*i for i in range(len(B))])+sum([C[i]*i for i in range(len(C))])))

    print('残りモノマー数:%f' % (sum(A)))
    plt.plot([C[i]*i for i in range(len(A))])
    plt.show()
    plt.plot([B[i]*i for i in range(len(B))])
    plt.show()
    

In [ ]:
sum([C[i]*i for i in range(N)])/len(C)